<a href="https://colab.research.google.com/github/Jaejuna/SpeedWagon/blob/main/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/train.csv')
valid = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/valid.csv')

In [4]:
train.head()

,Text,Summary,Topic
0,웃긴게 20년동안... 바로 옆 가게 주차장에서 같이 그 감자탕집 주차도 했었는데 ...,20년 동안 감자탕 집 주차도 했었는데 감자탕 집이 장사 잘 되니까 배 아파서 주차...,주거와생활
1,아까 보니깐 로켓 프레시 잇드라 #@시스템#사진# 나도 밥 무거요 호앵 야 제주도는...,제주시에 쿠팡 로켓 배송이나 택배가 오는지 물어보고 있다.,주거와생활
2,#@이름#아 먼가 나머리망 이불위에 두고 온거 같다 확인좀 ㅠ 읭?? 만약에 잇으면...,이불 위에 머리 망이 없다고 하니까 그럼 넣은 것 같다고 했다.,주거와생활
3,분양가 얼마여써? 여기 일억육천구백이더라ㅔ ㅋㅋㅋㅋㅋ걍 여기서 살고싶다..........,분양가 이억이 넘는데 일억은 대출로 받아야 한다.,주거와생활
4,지금 뭔가 카톡 느리지않녀 새해라 다들 톡하느라 터진건가 ㅇㅇ 약간 버벅댐 ㅇㅇㅇ ...,새해에 다들 카톡을 해서 그런지 카톡이 약간 느렸지만 괜찮아졌다.,주거와생활


In [5]:
valid.head()

,Text,Summary,Topic
0,수유로오셈^^ 지금? 지금가도대? 지금이겠냐? 되겠냐? 집 데려다주냐? 왜안댐? ㅋ...,지금 수유에 갈 테니 집에 데려다 달라고 한다.,행사
1,제주도가구시퍼져써 여보시간 업자나 2박3일정두 갔다올수이ㅛ을거같운데 쉬는날몰아서 구...,2박 3일 정도 쉬는 날을 몰아서 제주도에 갔다 오기로 했다.,행사
2,무튼 몇시에 어디서 만나지? 정한거 아냐? 메가박스 2시 45분 매표소?역출구? 매...,메가박스 매표소에서 2시 45분에 만나기로 약속했다.,행사
3,낼상황봐서 정하자 오켕 낼 보구 밥 정하쟈 ~ 웅 조아용 그리구 간지카페가자 힐링해...,내일 상황을 봐서 정하자고 카페를 가서 힐링해야 한다고 한다.,행사
4,내일 모레 머먹어? 김장김치랑 수육!!!!!!!!!! 수!!! 육!!!!! 나는? ...,모레 김장 김치와 수육을 먹을 테니 집으로 오라고 한다.,행사


## EDA

In [ ]:
print("Size of train data :", len(train))
print("Size of validation data :", len(valid))

Size of train data : 279992
Size of validation data : 35004


In [ ]:
# Train / Valid dataset split rate
len(train) / len(valid) # 8:1 비율

7.998857273454463

In [ ]:
# Topic 종류
train['Topic'].unique()

array(['주거와생활', '시사교육', '상거래(쇼핑)', '개인및관계',
       '행사', '미용과건강', '일과직업', '식음료', '여가생활'],
      dtype=object)

In [ ]:
valid['Topic'].unique()

array(['행사', '시사교육', '상거래(쇼핑)', '개인및관계',
       '주거와생활', '여가생활', '미용과건강', '일과직업',
       '식음료'], dtype=object)

In [ ]:
# Topic 별 Dialogue 수
pd.DataFrame(train.groupby("Topic").count().iloc[:,0])

,Text
Topic,
개인및관계,71130
미용과건강,17069
상거래(쇼핑),26298
시사교육,13119
식음료,30184
여가생활,35247
일과직업,20428
주거와생활,45179
행사,21338


In [ ]:
pd.DataFrame(valid.groupby("Topic").count().iloc[:,0])

,Text
Topic,
개인및관계,8892
미용과건강,2134
상거래(쇼핑),3288
시사교육,1640
식음료,3773
여가생활,4407
일과직업,2554
주거와생활,5648
행사,2668


## Data Preprocessing

In [7]:
# 네이버 맞춤법 검사기
!pip install git+https://github.com/jungin500/py-hanspell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jungin500/py-hanspell to /tmp/pip-req-build-tf74hdvd
  Running command git clone --filter=blob:none --quiet https://github.com/jungin500/py-hanspell /tmp/pip-req-build-tf74hdvd
  Resolved https://github.com/jungin500/py-hanspell to commit b4fb712334496821db4dffb981cc73a9bbaee673
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4808 sha256=315f5bf7a948de87d86795cec1f3bc2905fe09713fa887efaa22308482158f84
  Stored in directory: /tmp/pip-ephem-wheel-cache-gtn0a3k5/wheels/68/4f/41/c18b9dcfedfd7919c66badb0afcdc879b9d7a6123212c34b15
Successfully built py-hanspell


In [8]:
from hanspell import spell_checker
import re

In [21]:
def preprocess(text):
  text = text.lower() # 영어 텍스트를 소문자로 변환
  text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', text) # 반복되는 자음, 모음 제거 (ex. ㅋㅋ ㅠㅠ)
  text = spell_checker.check(text).checked # 맞춤법 검사
  text = re.sub(r'\.{3,}', '..', text) # .이 3개 이상 반복되면 2개로 변환 
  text = re.sub(r'\?{2,}', '?', text) # ?이 2개 이상 반복되면 1개로 변환
  text = re.sub(r'\!{2,}', '!', text) # !이 2개 이상 반복되면 1개로 변환
  text = re.sub(r'\~{2,}', '~', text) # ~이 2개 이상 반복되면 1개로 변환
  text = re.sub(r'[" "]+', " ", text) # 공백 여러 개를 1개로 변환
  text = text.strip() # 양쪽 공백 제거
  return text

In [17]:
train.iloc[2,0]

'#@이름#아 먼가 나머리망 이불위에 두고 온거 같다 확인좀 ㅠ 읭?? 만약에 잇으면 침대 밑 바닥에 놔줭 #@이모티콘# 기달 업는뎅ㅜ 구럼 내가 너엇나바 ㅋㅋㅋㅋ 땡큐'

In [18]:
preprocess(train.iloc[2,0])

'#@이름#아 먼가 나 머리망 이불 위에 두고 온 거 같다 확인 좀 ㅠ 응? 만약에 있으면 침대 밑 바닥에 놔줘 #@이모티콘# 기달 업는데ᅮ 그럼 내가 너 엇나 바 땡큐'

In [30]:
for i in range(50):
  print(preprocess(valid.iloc[i,0]))

수유로 오세요^^ 지금? 지금 가도 대? 지금이겠냐? 되겠냐? 집 데려다주냐? 왜 안댐? 네가 나 데려다줘야지 내가 수유 가는데
제주도가 구 시퍼죠 써 여보 시간 업자나 2박 3일 정도 갔다 올 수 이ㅛ을 거 같은데 쉬는 날 몰아서 구래? 갈수이셔? 웅갈수이띠 여유 생기 보라카이 가기 전 애 한번 갔다 오자 꾸래
아무튼 몇 시에 어디서 만나지? 정한 거 아냐? 메가박스 2시 45분 매표소? 역 출구? 매표소가 낫지 않을까? 역 출구 서있기 애매할걸 그래 와파 안 뜨면 연락 못 하니까 힘내서 찾아.. 왜 2시 45분인진 몰겠지만 그냥 3시 5분 영화니까 20분 전 힘내서 찾을게!
낼 상황 봐서 정하자 오케이 낼 보고 밥 정하자 ~ 웅 좋아요 그리고 간지 카페가 자 힐링해야 해 .. 힐링,, , 좋지~
내일모레 뭐 먹어? 김장김치랑 수육! 수! 육! 나는? 너도 와 그럼 나도 수육..! 어디로? ㅇ우리 집 가도 돼?..
온니 자나 잘 갔다 왕 자기는 제주도 숙소 예약하고 있음 전날까지 고통받네 빨리하고 자야지,,, 파이팅!
음.. 10분? 그럼 40분쯤 도착할 거 같아? 시간 봐서 카페 가려 구 커피.. 모닝커피.. ㅎ 응응 그 정도쯤 커피마시구있오 내가 그쪽으로 갈게ㅜ 응응
아니 그래서 내일은 뭘 하지 형부 전시? 음 갈 곳은 많아유 그럴까 아니면 닭발 ㄱ? 닭발 조와 치
나 오늘 피부과 가고 하느라 늦게 잘 거 같아서 그럼 오늘 만나긴 힘들겠지? 오늘 본가가서 자구 낼 드레스 투어 엄마랑 같이 움직일까 하고 아 윙윙 그래야겠다 오늘 피부과 가서 늦게 자구나 웅 오늘 어떻게 될지 전혀 모르겠어 응응 나도 ㅜ 설마 시간 되더라도 자기 집 가서 쉬어야 할 거 같기도 사인했다며~ 축하해 아 고마워! 계속 너무 바쁘다
몇 시까지 갈까? 글쎄 지금 상태로는 7시? 그래그래 좀 늦게 먹자~ 배가 안 고플 거 같아 그때 보자 응응!
정말 시간을 착각했어 2시간 전에 출발해야지 하구 3시에 출발한 거 같아 에고고 짐 오고 있어? 응 가구 있어 수고했어요.
님 시간은요 상관없댔

In [26]:
preprocess(train.iloc[334,0])

'#@이름#이야 난 왜 이리 피곤한 걸까.. 잠도 많이 자는데? 왜 그러지 정말.. 영양제 좀 먹어야 하나 난방기를 가스로 돌리는 건데 그거 땜에 그런 걸까? 가스? 가스난방기가 있어? 등유로 하는 거! 그건 머리 아플 텐데 냄새도 나고 근데 난방기가 이거밖에 없어 다른 거 설치해달라고 해 환경이 참..'

In [ ]:
train.Text = train.Text.map(lambda x: preprocess(x))

In [23]:
valid.Text = valid.Text.map(lambda x: preprocess(x))

ParseError: ignored

In [ ]:
train.head()

In [ ]:
valid.head()

https://velog.io/@jx7789/%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%86%A1-%EB%8C%80%ED%99%94-%EC%9A%94%EC%95%BD-%EC%84%9C%EB%B9%84%EC%8A%A4-pepkrzfe